In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import timeit

from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve
from sklearn.metrics import roc_auc_score, roc_curve, f1_score, precision_score, recall_score, accuracy_score, auc
from hmeasure import h_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, precision_recall_fscore_support,roc_auc_score, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline

import optuna

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
from keras.layers import Dense, Dropout
from keras.regularizers import l2
from keras.layers import LeakyReLU
from keras.wrappers.scikit_learn import KerasClassifier
from keras import initializers
from keras.models import Sequential
from keras.optimizers import Adam

# Read the data using csv
path_train = 'credit_risk-dataset-cleanedtraining.csv'
df_train = pd.read_csv(path_train, encoding = "ISO-8859-1")

path_test = 'credit_risk-dataset-cleanedtesting.csv'
df_test = pd.read_csv(path_test, encoding = "ISO-8859-1")

# Prin 5 forst lines of df
df_train.head()

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_cred_hist_length,person_home_ownership,loan_intent,loan_grade,cb_person_default_on_file
0,23,62500,7.0,26000,11.71,1,0.42,2,3,3,1,0
1,23,120000,1.0,25600,12.69,0,0.21,3,3,1,1,0
2,22,60000,0.0,25475,10.99,1,0.42,3,3,5,1,0
3,23,83000,7.0,25300,10.99,1,0.30,3,3,1,1,0
4,21,42500,3.0,25000,12.73,1,0.59,3,3,1,2,1


In [6]:
X_test, y_test = df_test.iloc[:,:-1], df_test.iloc[:,-1]

X_train, y_train = df_train.iloc[:,:-1], df_train.iloc[:,-1]

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, test_size=0.2)

In [7]:
class tuning_methods:
    
    def __init__(self, X_train, X_val, y_train,  y_val, X_test, y_test):
        
        self.X_train = X_train
        self.X_val = X_val
        self.X_test = X_test
        self.y_test = y_test
        self.y_train = y_train
        self.y_val = y_val
    
  # Define the Keras model
    def create_model(self, units_1, alpha_1, dropout_1, l2_kern_1, l2_bias_1,
                 units_2, alpha_2, dropout_2, l2_kern_2, l2_bias_2,
                 units_3, alpha_3, dropout_3, l2_kern_3, l2_bias_3,
                 lr, batch_size):        # 
    
        model = Sequential()
    
        model.add(Dense(units_1, 
                input_shape=(X_train.shape[1],), 
                activation = LeakyReLU(alpha=alpha_1),
                kernel_regularizer = l2(l2_kern_1),
                bias_regularizer = l2(l2_bias_1),
                kernel_initializer = initializers.RandomNormal(mean=0.0, stddev=0.05)
        ))
        model.add(Dropout(dropout_1))
    
        model.add(Dense(units_2,
                    activation =LeakyReLU(alpha=alpha_2),
                    kernel_regularizer = l2(l2_kern_2),
                    bias_regularizer = l2(l2_bias_2),
                    kernel_initializer = initializers.RandomNormal(mean=0.0, stddev=0.05)
        ))
        model.add(Dropout(dropout_2))
    
        model.add(Dense(units_3,
                        activation =LeakyReLU(alpha=alpha_3),
                        kernel_regularizer = l2(l2_kern_3),
                        bias_regularizer = l2(l2_bias_3),
                        kernel_initializer = initializers.RandomNormal(mean=0.0, stddev=0.05)
        ))
        model.add(Dropout(dropout_3))
    
        model.add(Dense(1, activation='sigmoid'))
    
        optimizer = Adam(learning_rate=lr)
        model.compile(optimizer=optimizer,
                  loss='binary_crossentropy', 
                  metrics=['AUC'])
        return model

    
    # Define the Optuna study
    def objective(self, trial):
        
        # Define the pipeline
        pipeline = Pipeline([
                ('scaler', StandardScaler()),
                ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))
                ])
        # Define hyperparameters range values for tuning
        units_1 = trial.suggest_int('units_1', 2, 40)
        units_2 = trial.suggest_int('units_2', 2, 40)
        units_3 = trial.suggest_int('units_3', 2, 40)
        alpha_1 = trial.suggest_float('alpha_1', 0.01, 0.1)
        alpha_2 = trial.suggest_float('alpha_2', 0.01, 0.1)
        alpha_3 = trial.suggest_float('alpha_3', 0.01, 0.1)
        dropout_1 = trial.suggest_float("dropout_1", 0.2, 0.5)    
        dropout_2 = trial.suggest_float("dropout_2", 0.2, 0.5)    
        dropout_3 = trial.suggest_float("dropout_3", 0.2, 0.5)    
        l2_kern_1 = trial.suggest_float("l2_kern_1",  1e-5, 1e-2, log=True)
        l2_kern_2 = trial.suggest_float("l2_kern_2",  1e-5, 1e-2, log=True)
        l2_kern_3 = trial.suggest_float("l2_kern_3",  1e-5, 1e-2, log=True)
        l2_bias_1 = trial.suggest_float("l2_bias_1",  1e-5, 1e-2, log=True)
        l2_bias_2 = trial.suggest_float("l2_bias_2",  1e-5, 1e-2, log=True)
        l2_bias_3 = trial.suggest_float("l2_bias_3",  1e-5, 1e-2, log=True)
        lr = trial.suggest_float("lr", 5e-5, 1e-2, log=True)
        batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128, 256, 512, 1024])
        epochs = 200
        validation_data = (self.X_val, self.y_val)
        early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
        
        # Set parameters into pipeline
        pipeline.set_params(classifier__units_1=units_1,
                        classifier__alpha_1=alpha_1,
                        classifier__l2_kern_1=l2_kern_1,
                        classifier__l2_bias_1=l2_bias_1,
                        classifier__dropout_1=dropout_1,
                        classifier__units_2=units_2,
                        classifier__alpha_2=alpha_2,
                        classifier__l2_kern_2=l2_kern_2,
                        classifier__l2_bias_2=l2_bias_2,
                        classifier__dropout_2=dropout_2,
                        classifier__units_3=units_3,
                        classifier__alpha_3=alpha_3,
                        classifier__l2_kern_3=l2_kern_3,
                        classifier__l2_bias_3=l2_bias_3,
                        classifier__dropout_3=dropout_3,
                        classifier__lr=lr,
                        classifier__epochs = epochs,
                        classifier__batch_size = batch_size,
                        classifier__validation_data = validation_data,
                        classifier__callbacks=[early_stopping])
        
        # Apply cross validation
        kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=2023)
        scores = cross_val_score(pipeline, self.X_train, self.y_train, cv=kfold, scoring="roc_auc", error_score='raise')
    
        if trial.should_prune():
                raise optuna.TrialPruned()

        return np.mean(scores)

    def Tuning(self, n_trials=100, sampler = optuna.samplers.TPESampler()):
        
        start = timeit.default_timer()

        study = optuna.create_study(direction="maximize",             
                            sampler=sampler, 
                            pruner= optuna.pruners.MedianPruner()
                           )
        study.optimize(self.objective, n_trials=n_trials)  # to be converged it needs atleast 200 trials 

        print("Number of finished trials: {}".format(len(study.trials)))

        print("Best trial:")
        trial = study.best_trial

        print("  Value: {}".format(trial.value))

        stop = timeit.default_timer()

        time_opt = stop - start
        params_TPE = []

        for key, value in trial.params.items():
            params_TPE.append([key,value])
            print("    {}: {}".format(key, value))
            
        params_TPE.append(["time",round(time_opt/60,2)])

        return params_TPE
    
    def Best_MLP_model(self, params):
        
        # Define hyperpameters
        units_1 = params[0][1]
        units_2 = params[1][1]
        units_3 = params[2][1]
        alpha_1 = np.round(params[3][1],3)
        alpha_2 = np.round(params[4][1],3)
        alpha_3 = np.round(params[5][1],3)
        dropout_1 = np.round(params[6][1],2)
        dropout_2 = np.round(params[7][1],2)
        dropout_3 = np.round(params[8][1],2)
        kernel_regularizer_1=np.round(params[9][1],5)
        kernel_regularizer_2=np.round(params[10][1],5)
        kernel_regularizer_3=np.round(params[11][1],5)
        bias_regularizer_1=np.round(params[12][1],5)
        bias_regularizer_2=np.round(params[13][1],5)
        bias_regularizer_3=np.round(params[14][1],5)
        lr = np.round(params[15][1],5)
        batchs = params[16][1]
    
        # Define the pipeline
        pipeline = Pipeline([
                    ('scaler', StandardScaler()),
                    ('classifier', KerasClassifier(build_fn=tun.create_model, verbose=0))
                ])

        # Set hyperparameters
        epochs = 200
        validation_data = (self.X_val, self.y_val)
        early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
        pipeline.set_params(classifier__units_1=units_1,
                            classifier__alpha_1 = alpha_1,
                            classifier__l2_kern_1=kernel_regularizer_1,
                            classifier__l2_bias_1=bias_regularizer_1,
                            classifier__dropout_1=dropout_1,
                            classifier__units_2=units_2,
                            classifier__alpha_2=alpha_2,
                            classifier__l2_kern_2=kernel_regularizer_2,
                            classifier__l2_bias_2=bias_regularizer_2,
                            classifier__dropout_2=dropout_2,
                            classifier__units_3=units_3,
                            classifier__alpha_3=alpha_3,
                            classifier__l2_kern_3=kernel_regularizer_3,
                            classifier__l2_bias_3=bias_regularizer_3,
                            classifier__dropout_3=dropout_3,
                            classifier__lr=lr,
                            classifier__epochs = epochs,
                            classifier__batch_size = batchs,
                            classifier__validation_data = validation_data,
                            classifier__callbacks=[early_stopping])
        
        # Use cross validation to monitor the deferrence between training and testing data as an evidence of overfitting
        kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2023)
        scores_train = cross_val_score(pipeline, self.X_train, self.y_train, cv=kfold, scoring="roc_auc", error_score='raise')
        mean_score_train = np.mean(scores_train)

        kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2023)
        scores_test = cross_val_score(pipeline, self.X_test, self.y_test, cv=kfold, scoring="roc_auc", error_score='raise')
        mean_score_test = np.mean(scores_test)
        print(f"Train mean AUC score is {mean_score_train} while test mean AUC score is {mean_score_test}.")
      
        # Fit the pipeline to the training data
        history = pipeline.fit(self.X_train, self.y_train)

        #score = pipeline.evaluate(X_test, y_test, verbose=0)  

        y_pred_prob_tpe = pipeline.predict(self.X_test)

        y_pred_tpe = y_pred_prob_tpe.round()

        print(classification_report(self.y_test, y_pred_tpe, target_names = ['No','Yes']))

        acc = accuracy_score(self.y_test, y_pred_tpe)
        pre = precision_score(self.y_test, y_pred_tpe)
        rec = recall_score(self.y_test, y_pred_tpe)
        f1 = f1_score(self.y_test, y_pred_tpe)
        auc = roc_auc_score(self.y_test, y_pred_tpe)
        
        y_test_array = np.array(self.y_test)
        y_pred_tpe_array = np.array(y_pred_tpe)
        H_measure = h_score(y_test_array, y_pred_tpe_array)

        opt = [acc, pre, rec, f1, auc, H_measure]

        scores = pd.DataFrame(opt, index =['Accuracy', 'Precision', 'Recall', 'F1-score', 'AUC', "H-measure"], 
                              columns =['Tree-structured Parzen Estimator'])
        
        return scores 

In [12]:
tun = tuning_methods(X_train, X_val, y_train,  y_val, X_test, y_test)
trials = 10

# Tree-structured Parzen Estimator
params_TPE = tun.Tuning(n_trials=trials, sampler = optuna.samplers.TPESampler())

[I 2024-12-01 15:30:42,846] A new study created in memory with name: no-name-a301ee6c-5284-4eba-ac6c-f951edc813dd
C:\Users\navee\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:60: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


195/195 [==============================] - 0s 1ms/step


[I 2024-12-01 15:31:41,914] Trial 0 finished with value: 0.8977993985489211 and parameters: {'units_1': 11, 'units_2': 17, 'units_3': 33, 'alpha_1': 0.09465647988800066, 'alpha_2': 0.055046441086712763, 'alpha_3': 0.03160754959679402, 'dropout_1': 0.4794835023897406, 'dropout_2': 0.2627698059165294, 'dropout_3': 0.27172189330972335, 'l2_kern_1': 2.5949609366162603e-05, 'l2_kern_2': 0.00015717373066867477, 'l2_kern_3': 9.882898312750379e-05, 'l2_bias_1': 9.83880485782341e-05, 'l2_bias_2': 7.819880675895657e-05, 'l2_bias_3': 0.005563504099048016, 'lr': 0.00046154899437148803, 'batch_size': 128}. Best is trial 0 with value: 0.8977993985489211.
C:\Users\navee\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:60: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


195/195 [==============================] - 0s 2ms/step


[I 2024-12-01 15:32:27,488] Trial 1 finished with value: 0.8980080310816273 and parameters: {'units_1': 22, 'units_2': 6, 'units_3': 30, 'alpha_1': 0.042288936539744186, 'alpha_2': 0.03047613190205814, 'alpha_3': 0.02407235236624014, 'dropout_1': 0.2278603990131359, 'dropout_2': 0.35225032548193164, 'dropout_3': 0.4249472923124793, 'l2_kern_1': 0.0015013299930945123, 'l2_kern_2': 9.574909236319662e-05, 'l2_kern_3': 0.0005407669982463723, 'l2_bias_1': 4.4613200805419135e-05, 'l2_bias_2': 1.7937718085781485e-05, 'l2_bias_3': 0.0020392538320976937, 'lr': 0.00026339321748638974, 'batch_size': 512}. Best is trial 1 with value: 0.8980080310816273.
C:\Users\navee\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:60: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


...

195/195 [==============================] - 0s 2ms/step


[I 2024-12-01 15:44:13,296] Trial 8 finished with value: 0.8925281381317922 and parameters: {'units_1': 35, 'units_2': 12, 'units_3': 7, 'alpha_1': 0.024589702525163515, 'alpha_2': 0.05810767891950795, 'alpha_3': 0.05633433320594007, 'dropout_1': 0.4973657659632869, 'dropout_2': 0.28422764236073517, 'dropout_3': 0.3588814892845874, 'l2_kern_1': 0.0019919098049209763, 'l2_kern_2': 0.0011533460183812344, 'l2_kern_3': 0.005732586454387019, 'l2_bias_1': 0.0029274967457332884, 'l2_bias_2': 0.006029555184657117, 'l2_bias_3': 0.00024193740413089337, 'lr': 7.5897697907576e-05, 'batch_size': 1024}. Best is trial 3 with value: 0.8990319103744762.
C:\Users\navee\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:60: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


195/195 [==============================] - 0s 2ms/step


[I 2024-12-01 15:44:30,768] Trial 9 finished with value: 0.8996270028784122 and parameters: {'units_1': 21, 'units_2': 8, 'units_3': 2, 'alpha_1': 0.03263648113236906, 'alpha_2': 0.0630450102940889, 'alpha_3': 0.019127218785505204, 'dropout_1': 0.21897778267573278, 'dropout_2': 0.3224446852123366, 'dropout_3': 0.4478933455539782, 'l2_kern_1': 0.0001407750500934896, 'l2_kern_2': 0.0002551825705565549, 'l2_kern_3': 1.7690626429505184e-05, 'l2_bias_1': 5.730559669715484e-05, 'l2_bias_2': 0.0072230569503712524, 'l2_bias_3': 0.00040981958343761497, 'lr': 0.006536493097806165, 'batch_size': 1024}. Best is trial 9 with value: 0.8996270028784122.


Number of finished trials: 10
Best trial:
  Value: 0.8996270028784122
    units_1: 21
    units_2: 8
    units_3: 2
    alpha_1: 0.03263648113236906
    alpha_2: 0.0630450102940889
    alpha_3: 0.019127218785505204
    dropout_1: 0.21897778267573278
    dropout_2: 0.3224446852123366
    dropout_3: 0.4478933455539782
    l2_kern_1: 0.0001407750500934896
    l2_kern_2: 0.0002551825705565549
    l2_kern_3: 1.7690626429505184e-05
    l2_bias_1: 5.730559669715484e-05
    l2_bias_2: 0.0072230569503712524
    l2_bias_3: 0.00040981958343761497
    lr: 0.006536493097806165
    batch_size: 1024


In [13]:
tpe_scores = tun.Best_MLP_model(params_TPE)

C:\Users\navee\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:169: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


37/37 [==============================] - 0s 2ms/step
Train mean AUC score is 0.899281190867778 while test mean AUC score is 0.893223674244471.
183/183 [==============================] - 0s 984us/step
              precision    recall  f1-score   support

          No       0.83      1.00      0.90      4814
         Yes       0.46      0.01      0.02      1021

    accuracy                           0.82      5835
   macro avg       0.65      0.50      0.46      5835
weighted avg       0.76      0.82      0.75      5835



In [14]:
# Random Sampler
trials = 10
params_rs = tun.Tuning(n_trials=trials, sampler = optuna.samplers.RandomSampler())
rs_scores = tun.Best_MLP_model(params_rs)
rs_scores = rs_scores.rename(columns={'Tree-structured Parzen Estimator': 'Random Sampler'})

[I 2024-12-01 15:45:23,458] A new study created in memory with name: no-name-b60921ad-596e-4aec-828a-752e8da6a5ff
C:\Users\navee\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:60: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


195/195 [==============================] - 0s 1ms/step


[I 2024-12-01 15:45:56,124] Trial 0 finished with value: 0.89857606429797 and parameters: {'units_1': 36, 'units_2': 33, 'units_3': 26, 'alpha_1': 0.010164818202157052, 'alpha_2': 0.04623286173233249, 'alpha_3': 0.08858909847133103, 'dropout_1': 0.48932583680930464, 'dropout_2': 0.36774045854624104, 'dropout_3': 0.4296449882091301, 'l2_kern_1': 0.0016134106763473026, 'l2_kern_2': 1.1531425411212442e-05, 'l2_kern_3': 0.007268073994994258, 'l2_bias_1': 0.0036663549560223586, 'l2_bias_2': 0.00023299132666956124, 'l2_bias_3': 0.00024749647140453427, 'lr': 0.0006727076108484775, 'batch_size': 1024}. Best is trial 0 with value: 0.89857606429797.
C:\Users\navee\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:60: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


195/195 [==============================] - 0s 1ms/step


[I 2024-12-01 15:46:31,810] Trial 1 finished with value: 0.8987237662888621 and parameters: {'units_1': 30, 'units_2': 27, 'units_3': 37, 'alpha_1': 0.04609227276161123, 'alpha_2': 0.014734844534838122, 'alpha_3': 0.0568148171622379, 'dropout_1': 0.26900843980151734, 'dropout_2': 0.3099765857504585, 'dropout_3': 0.3216849032636249, 'l2_kern_1': 0.0005439579098872581, 'l2_kern_2': 0.003014714962381012, 'l2_kern_3': 8.256341452696815e-05, 'l2_bias_1': 3.3058270396711655e-05, 'l2_bias_2': 1.1269769682458114e-05, 'l2_bias_3': 0.0003370012358255037, 'lr': 0.0005816146787295745, 'batch_size': 256}. Best is trial 1 with value: 0.8987237662888621.
C:\Users\navee\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:60: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


...

195/195 [==============================] - 0s 1ms/step


[I 2024-12-01 15:56:00,680] Trial 9 finished with value: 0.89872582835625 and parameters: {'units_1': 22, 'units_2': 36, 'units_3': 12, 'alpha_1': 0.06611105681540107, 'alpha_2': 0.07331687690921421, 'alpha_3': 0.08199469415260333, 'dropout_1': 0.41116878196394535, 'dropout_2': 0.4397676380641242, 'dropout_3': 0.4607655952911267, 'l2_kern_1': 1.5152040426844547e-05, 'l2_kern_2': 0.008199805906448879, 'l2_kern_3': 3.331153520331828e-05, 'l2_bias_1': 0.00027332329452065213, 'l2_bias_2': 0.0061773978161703885, 'l2_bias_3': 0.002661582920378538, 'lr': 0.0001690652863742106, 'batch_size': 16}. Best is trial 6 with value: 0.8990403277373776.


Number of finished trials: 10
Best trial:
  Value: 0.8990403277373776
    units_1: 27
    units_2: 34
    units_3: 15
    alpha_1: 0.034718122308698314
    alpha_2: 0.06977437018865705
    alpha_3: 0.04802064246146196
    dropout_1: 0.4752612608620585
    dropout_2: 0.48384801096051877
    dropout_3: 0.406912932068463
    l2_kern_1: 0.00014618971645773417
    l2_kern_2: 3.438038065450949e-05
    l2_kern_3: 0.005077143773130355
    l2_bias_1: 0.0008726063645589794
    l2_bias_2: 0.00011533891035197285
    l2_bias_3: 3.3857831615553653e-05
    lr: 0.0009420874595835336
    batch_size: 128


C:\Users\navee\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:169: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


37/37 [==============================] - 0s 2ms/step
Train mean AUC score is 0.8975351304148829 while test mean AUC score is 0.8920644986753409.
183/183 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

          No       0.89      0.89      0.89      4814
         Yes       0.50      0.49      0.50      1021

    accuracy                           0.82      5835
   macro avg       0.69      0.69      0.69      5835
weighted avg       0.82      0.82      0.82      5835



In [15]:
# Genetic Algorithm
params_ga = tun.Tuning(n_trials=trials, sampler = optuna.samplers.NSGAIISampler())
ga_scores = tun.Best_MLP_model(params_ga)
ga_scores = ga_scores.rename(columns={'Tree-structured Parzen Estimator': 'Genetic Algorithm'})

[I 2024-12-01 15:58:51,413] A new study created in memory with name: no-name-e3da3a87-0eab-4086-a011-2d35f4df07f4
C:\Users\navee\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:60: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


195/195 [==============================] - 1s 1ms/step


[I 2024-12-01 16:00:14,270] Trial 0 finished with value: 0.8983063402042326 and parameters: {'units_1': 12, 'units_2': 36, 'units_3': 36, 'alpha_1': 0.07195114156633683, 'alpha_2': 0.03499321902355449, 'alpha_3': 0.011689464291570095, 'dropout_1': 0.25201242154092235, 'dropout_2': 0.4290905450165321, 'dropout_3': 0.35824766555990073, 'l2_kern_1': 0.0002997561726895135, 'l2_kern_2': 0.002230259501377773, 'l2_kern_3': 0.0004312898465415555, 'l2_bias_1': 0.0003748062497994044, 'l2_bias_2': 0.000618144638434211, 'l2_bias_3': 0.006229838364059696, 'lr': 0.0009406244465523609, 'batch_size': 64}. Best is trial 0 with value: 0.8983063402042326.
C:\Users\navee\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:60: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


195/195 [==============================] - 0s 1ms/step


[I 2024-12-01 16:01:08,774] Trial 1 finished with value: 0.8982944726880402 and parameters: {'units_1': 9, 'units_2': 17, 'units_3': 11, 'alpha_1': 0.08899481587572336, 'alpha_2': 0.061981603405611055, 'alpha_3': 0.06712850610560706, 'dropout_1': 0.33578268291202407, 'dropout_2': 0.2514761010718352, 'dropout_3': 0.2250234026637089, 'l2_kern_1': 8.06256316596477e-05, 'l2_kern_2': 1.0841391818327202e-05, 'l2_kern_3': 0.004223834251076415, 'l2_bias_1': 6.116617738341529e-05, 'l2_bias_2': 0.005914776031414456, 'l2_bias_3': 0.0006831903560851215, 'lr': 0.00026485703901087105, 'batch_size': 1024}. Best is trial 0 with value: 0.8983063402042326.
C:\Users\navee\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:60: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


...

195/195 [==============================] - 1s 2ms/step


[I 2024-12-01 16:16:45,757] Trial 9 finished with value: 0.8988393394161144 and parameters: {'units_1': 29, 'units_2': 38, 'units_3': 28, 'alpha_1': 0.06933233456874978, 'alpha_2': 0.0501200421994091, 'alpha_3': 0.06312705096549352, 'dropout_1': 0.46796460606426543, 'dropout_2': 0.2736341271547967, 'dropout_3': 0.46422910111566457, 'l2_kern_1': 0.00011715964470462116, 'l2_kern_2': 0.0007944776402110861, 'l2_kern_3': 0.00015810322887134274, 'l2_bias_1': 0.00781361591103131, 'l2_bias_2': 0.0024239967316519945, 'l2_bias_3': 0.0006759873481765929, 'lr': 0.00030909832583910223, 'batch_size': 32}. Best is trial 9 with value: 0.8988393394161144.


Number of finished trials: 10
Best trial:
  Value: 0.8988393394161144
    units_1: 29
    units_2: 38
    units_3: 28
    alpha_1: 0.06933233456874978
    alpha_2: 0.0501200421994091
    alpha_3: 0.06312705096549352
    dropout_1: 0.46796460606426543
    dropout_2: 0.2736341271547967
    dropout_3: 0.46422910111566457
    l2_kern_1: 0.00011715964470462116
    l2_kern_2: 0.0007944776402110861
    l2_kern_3: 0.00015810322887134274
    l2_bias_1: 0.00781361591103131
    l2_bias_2: 0.0024239967316519945
    l2_bias_3: 0.0006759873481765929
    lr: 0.00030909832583910223
    batch_size: 32


C:\Users\navee\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:169: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


37/37 [==============================] - 0s 2ms/step
Train mean AUC score is 0.8983278302303506 while test mean AUC score is 0.8915245800287165.
183/183 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

          No       0.89      0.91      0.90      4814
         Yes       0.50      0.45      0.47      1021

    accuracy                           0.83      5835
   macro avg       0.69      0.68      0.68      5835
weighted avg       0.82      0.83      0.82      5835



In [16]:
tun = tuning_methods(X_train, X_val, y_train,  y_val, X_test, y_test)
trials = 10


In [17]:
# Quasi Monte Carlo
params_mc = tun.Tuning(n_trials=trials, sampler = optuna.samplers.QMCSampler())
mc_scores = tun.Best_MLP_model(params_mc)
mc_scores = mc_scores.rename(columns={'Tree-structured Parzen Estimator': 'Quasi Monte Carlo'})

C:\Users\navee\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: ExperimentalWarning: QMCSampler is experimental (supported from v3.0.0). The interface can change in the future.
  
[I 2024-12-01 16:27:06,241] A new study created in memory with name: no-name-3b926bcf-524f-4690-8564-4c2dba777646
C:\Users\navee\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:60: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


195/195 [==============================] - 0s 2ms/step


[I 2024-12-01 16:30:39,405] Trial 0 finished with value: 0.898192887664888 and parameters: {'units_1': 22, 'units_2': 36, 'units_3': 12, 'alpha_1': 0.05376935549508785, 'alpha_2': 0.044778660745382, 'alpha_3': 0.0880970788801615, 'dropout_1': 0.3988641105074193, 'dropout_2': 0.2403637909830827, 'dropout_3': 0.4923795751680284, 'l2_kern_1': 0.0004398238352361431, 'l2_kern_2': 0.0027554286349768245, 'l2_kern_3': 2.8786037994532855e-05, 'l2_bias_1': 0.0030639534284330728, 'l2_bias_2': 5.6402641106521115e-05, 'l2_bias_3': 0.0011749562487099014, 'lr': 7.019816909685598e-05, 'batch_size': 64}. Best is trial 0 with value: 0.898192887664888.
C:\Users\navee\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:60: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
[W 2024-12-01 16:30:39,416] The parameter 'l2_kern_1' in trial#1 is sampled in

195/195 [==============================] - 0s 2ms/step


[I 2024-12-01 16:35:33,687] Trial 1 finished with value: 0.8943987526936564 and parameters: {'units_1': 2, 'units_2': 2, 'units_3': 2, 'alpha_1': 0.01, 'alpha_2': 0.01, 'alpha_3': 0.01, 'dropout_1': 0.2, 'dropout_2': 0.2, 'dropout_3': 0.2, 'l2_kern_1': 0.00930718891865643, 'l2_kern_2': 2.2896929134203582e-05, 'l2_kern_3': 1.5127589370560652e-05, 'l2_bias_1': 9.808507153153919e-05, 'l2_bias_2': 0.00022871641708819003, 'l2_bias_3': 0.0008584642917158349, 'lr': 5.000000000000004e-05, 'batch_size': 32}. Best is trial 0 with value: 0.898192887664888.
C:\Users\navee\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:60: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
[W 2024-12-01 16:35:33,695] The parameter 'batch_size' in trial#2 is sampled independently by using `RandomSampler` instead of `QMCSampler` (optimization performance ma

...

195/195 [==============================] - 0s 2ms/step


[I 2024-12-01 16:49:16,679] Trial 8 finished with value: 0.8974451663711043 and parameters: {'units_1': 6, 'units_2': 26, 'units_3': 16, 'alpha_1': 0.02125, 'alpha_2': 0.02125, 'alpha_3': 0.043750000000000004, 'dropout_1': 0.3875, 'dropout_2': 0.3875, 'dropout_3': 0.3875, 'l2_kern_1': 0.004216965034285825, 'l2_kern_2': 0.0007498942093324562, 'l2_kern_3': 2.3713737056616547e-05, 'l2_bias_1': 0.0007498942093324562, 'l2_bias_2': 0.00013335214321633237, 'l2_bias_3': 2.3713737056616547e-05, 'lr': 9.696137237434293e-05, 'batch_size': 16}. Best is trial 7 with value: 0.8991453561280406.
C:\Users\navee\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:60: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
[W 2024-12-01 16:49:16,688] The parameter 'batch_size' in trial#9 is sampled independently by using `RandomSampler` instead of `QMCSa

195/195 [==============================] - 0s 1ms/step


[I 2024-12-01 16:49:42,264] Trial 9 finished with value: 0.8972115888764866 and parameters: {'units_1': 9, 'units_2': 14, 'units_3': 38, 'alpha_1': 0.04937500000000001, 'alpha_2': 0.060625000000000005, 'alpha_3': 0.038125000000000006, 'dropout_1': 0.33125000000000004, 'dropout_2': 0.48125, 'dropout_3': 0.48125, 'l2_kern_1': 8.659643233600651e-05, 'l2_kern_2': 0.0011547819846894588, 'l2_kern_3': 1.5399265260594915e-05, 'l2_bias_1': 0.0064938163157621165, 'l2_bias_2': 0.0064938163157621165, 'l2_bias_3': 0.0027384196342643626, 'lr': 0.007181011550336231, 'batch_size': 512}. Best is trial 7 with value: 0.8991453561280406.
C:\Users\navee\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:169: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


Number of finished trials: 10
Best trial:
  Value: 0.8991453561280406
    units_1: 26
    units_2: 6
    units_3: 36
    alpha_1: 0.06625
    alpha_2: 0.06625
    alpha_3: 0.08875000000000001
    dropout_1: 0.23750000000000002
    dropout_2: 0.23750000000000002
    dropout_3: 0.23750000000000002
    l2_kern_1: 0.00013335214321633237
    l2_kern_2: 2.3713737056616547e-05
    l2_kern_3: 0.0007498942093324562
    l2_bias_1: 2.3713737056616547e-05
    l2_bias_2: 0.004216965034285825
    l2_bias_3: 0.0007498942093324562
    lr: 0.0013712408783810375
    batch_size: 16
37/37 [==============================] - 0s 3ms/step
Train mean AUC score is 0.9003665168702112 while test mean AUC score is 0.8929107550595503.
183/183 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

          No       0.83      0.99      0.90      4814
         Yes       0.45      0.04      0.07      1021

    accuracy                           0.82      5835
   macro avg

In [18]:
all_scores = pd.concat([tpe_scores, rs_scores, ga_scores, mc_scores], axis =1)
all_scores

,Tree-structured Parzen Estimator,Random Sampler,Genetic Algorithm,Quasi Monte Carlo
Accuracy,0.824679,0.823650,0.825364,0.823650
Precision,0.464286,0.496071,0.501101,0.451220
Recall,0.012733,0.494613,0.445642,0.036239
F1-score,0.024786,0.495341,0.471747,0.067090
AUC,0.504808,0.694024,0.675770,0.513446
H-measure,0.004285,0.232129,0.207285,0.011715


In [20]:
all_scores.to_csv('Credit_risk_dataset_3_Layer_scores.csv', index=False)